##### Valores de mercado en descenso

Con este código podrás descargar la lista con las mayores bajadas de valor de mercado ofrecida por transfermarkt (al menos, a día 02/05/2023)

Importamos las librerías necesarias

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import datetime as dt
import time
import numpy as np
import pandas as pd
import openpyxl

Leemos la página con Beautiful Soup

In [2]:
url = 'https://www.transfermarkt.es/spieler-statistik/marktwertverluste/marktwertetop'
headers = {"User-Agent":"Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = bs(response.text, 'lxml')

Vemos sobre cuántas páginas vamos a iterar (todas las que haya)

In [ ]:
numero_paginas = int(soup.find_all('li', class_='tm-pagination__list-item')[-1].a['href'].split('=')[1])
numero_paginas

Creamos el bucle para obtener toda la información.

In [ ]:
nombres = []
posicion = []
equipo = []
nacionalidad_1 = []
nacionalidad_2 = []
edad=[]
valor_mercado =[]
porcentaje_subida = []
diferencia = []
lista_urls = []

url_principal = 'https://www.transfermarkt.es'

for page in range(1, numero_paginas+1):
    time.sleep(1)
    url_pagina = url + "?page=" + str(page)
    headers = {"User-Agent":"Mozilla/5.0"}
    response = requests.get(url_pagina, headers=headers)
    soup_pagina = bs(response.text, 'lxml')

    print('Página: ' + str(page))

    contenedor_jugador_1 = soup_pagina.find_all('tr', class_='odd')
    contenedor_jugador_2 = soup_pagina.find_all('tr', class_='even')

    for jugador_1 in contenedor_jugador_1:
        
        #NOMBRE
        try:
            nombres.append(jugador_1.find('td', class_='hauptlink').get_text(strip=True))
        except:
            nombres.append('No veo el nombre')

        #POSICIÓN
        try:
            posicion.append(jugador_1.find_all('tr')[1].get_text(strip=True))
        except:
            posicion.append('No veo la posicion')

        #EQUIPO
        try:
            equipo.append(jugador_1.find_all('img')[1]['alt'])
        except:
            equipo.append('No veo el equipo')

        #NACIONALIDAD

        try:
            contenedor_nacion = jugador_1.find_all('td', class_='zentriert')[2]
            nacionalidades = contenedor_nacion.find_all('img')
            
            if len(nacionalidades) >= 2:
                nacionalidad_1.append(str(nacionalidades[0]['alt']))
                nacionalidad_2.append(nacionalidades[1]['alt'])
            else:
                nacionalidad_1.append(nacionalidades[0]['alt'])
                nacionalidad_2.append('No tiene')
        except:
            nacionalidad_1.append('No veo nacionalidad')
            nacionalidad_2.append('No veo nacionalidad')

        #EDAD

        try:
            edad.append(int(jugador_1.find_all('td', class_='zentriert')[3].get_text(strip=True)))
        except:
            edad.append('No veo edad')

        #VALOR DE MERCADO
        try:
            if 'mill' in jugador_1.find('td',class_='rechts hauptlink').get_text():
                valor_mercado.append(int((jugador_1.find('td',class_='rechts hauptlink').get_text().split()[0].replace(',','')) + '0000'))
            elif 'mil' in jugador_1.find('td',class_='rechts hauptlink').get_text():
                valor_mercado.append(int((jugador_1.find('td',class_='rechts hauptlink').get_text().split()[0].replace(',','')) + '000'))
            else:
                valor_mercado.append(jugador_1.find('td',class_='rechts hauptlink').get_text())
        
        except:
            valor_mercado.append('No veo valor')

        #PORCENTAJE DE SUBIDA
        try:
            porcentaje_subida.append(jugador_1.find_all('td', class_='rechts')[1].get_text())
        except:
            porcentaje_subida.append('No veo subida/bajada')

        #SUBIDA
        try:
            diferencia.append(jugador_1.find_all('td', class_='rechts')[2].get_text())
        except:
            diferencia.append('No veo diferencia')

        #URLS
        try:
            lista_urls.append(url_principal + jugador_1.find('a')['href'])
        except:
            lista_urls.append('No veo url')

        
    
    
    for jugador_2 in contenedor_jugador_2:

        #NOMBRES
        try:
            nombres.append(jugador_2.find('td', class_='hauptlink').get_text(strip=True))
        except:
            nombres.append('No veo el nombre')
        
        #POSICIÓN
        try:
            posicion.append(jugador_2.find_all('tr')[1].get_text(strip=True))
        except:
            posicion.append('No veo la posicion')

        #EQUIPO
        try:
            equipo.append(jugador_2.find_all('img')[1]['alt'])
        except:
            equipo.append('No veo el equipo')

        #NACIONALIDAD
        try:
            contenedor_nacion = jugador_2.find_all('td', class_='zentriert')[2]
            nacionalidades = contenedor_nacion.find_all('img')
            if len(nacionalidades) >= 2:
                nacionalidad_1.append(str(nacionalidades[0]['alt']))
                nacionalidad_2.append(nacionalidades[1]['alt'])
            else:
                nacionalidad_1.append(nacionalidades[0]['alt'])
                nacionalidad_2.append('No tiene')
        except:
            nacionalidad_1.append('No veo nacionalidad')
            nacionalidad_2.append('No veo nacionalidad')
        
        #EDAD
        try:
            edad.append(int(jugador_2.find_all('td', class_='zentriert')[3].get_text(strip=True)))
        except:
            edad.append('No veo edad')

        #VALOR DE MERCADO
        try:
            if 'mill' in jugador_2.find('td',class_='rechts hauptlink').get_text():
                valor_mercado.append(int((jugador_2.find('td',class_='rechts hauptlink').get_text().split()[0].replace(',','')) + '0000'))
            elif 'mil' in jugador_2.find('td',class_='rechts hauptlink').get_text():
                valor_mercado.append(int((jugador_2.find('td',class_='rechts hauptlink').get_text().split()[0].replace(',','')) + '000'))
            else:
                valor_mercado.append(jugador_2.find('td',class_='rechts hauptlink').get_text())
        
        except:
            valor_mercado.append('No veo valor')
        
         #PORCENTAJE DE SUBIDA
        try:
            porcentaje_subida.append(jugador_2.find_all('td', class_='rechts')[1].get_text())
        except:
            porcentaje_subida.append('No veo subida/bajada')

        #SUBIDA
        try:
            diferencia.append(jugador_2.find_all('td', class_='rechts')[2].get_text())
        except:
            diferencia.append('No veo diferencia')

        #URLS
        try:
            lista_urls.append(url_principal + jugador_2.find('a')['href'])
        except:
            lista_urls.append('No veo url')

Creamos el DataFrame

In [5]:
lista_df = []

for i in range(len(nombres)):
    data = {'Fecha de actualización': dt.datetime.now().strftime('%d/%m/%Y'),
            'Hora de actualización': dt.datetime.now().strftime('%H:%M'),
        'Nombre': nombres[i],
        'Posición':posicion[i],
        'Equipo': equipo[i],
        'Nacionalidad 1': nacionalidad_1[i],
        'Nacionalidad 2': nacionalidad_2[i],
        'Edad': edad[i],
        'Valor de Mercado':valor_mercado[i],
        'Porcentaje de subida':porcentaje_subida[i],
        'Subida/bajada':diferencia[i],
        'Url del jugador':lista_urls[i]
        
        }
    lista_df.append(data)

df = pd.DataFrame(lista_df)

Exportamos el DataFrame a Excel

In [7]:
df.to_excel('../data/ValoresDeMercado_Bajadas.xlsx')

##### Ahora
Queremos obtener los links a los valores detallados de cada jugador 

In [12]:
valores_detallados = []
contador = 1

for url_jugador in df['Url del jugador']:
    
    print('Jugador ' +str(contador) +' de '+str(3977))
    contador+=1
    headers = {"User-Agent":"Mozilla/5.0"}
    response = requests.get(url_jugador, headers=headers)
    soup_jugador = bs(response.text, 'lxml')
   

    links = soup_jugador.find_all('a')

    for link in links:
        try:
            if 'Ver página de valores det' in link.get_text():
                valores_detallados.append(url_principal + link['href'])
            else:
                valores_detallados.append('No veo la página')
        except:
            valores_detallados.append('No veo la página')

Jugador 1 de 3977
Jugador 2 de 3977
Jugador 3 de 3977
Jugador 4 de 3977
Jugador 5 de 3977
Jugador 6 de 3977
Jugador 7 de 3977
Jugador 8 de 3977
Jugador 9 de 3977
Jugador 10 de 3977
Jugador 11 de 3977
Jugador 12 de 3977
Jugador 13 de 3977
Jugador 14 de 3977
Jugador 15 de 3977
Jugador 16 de 3977
Jugador 17 de 3977
Jugador 18 de 3977
Jugador 19 de 3977
Jugador 20 de 3977
Jugador 21 de 3977
Jugador 22 de 3977
Jugador 23 de 3977
Jugador 24 de 3977
Jugador 25 de 3977
Jugador 26 de 3977
Jugador 27 de 3977
Jugador 28 de 3977
Jugador 29 de 3977
Jugador 30 de 3977
Jugador 31 de 3977
Jugador 32 de 3977
Jugador 33 de 3977
Jugador 34 de 3977
Jugador 35 de 3977
Jugador 36 de 3977
Jugador 37 de 3977
Jugador 38 de 3977
Jugador 39 de 3977
Jugador 40 de 3977
Jugador 41 de 3977
Jugador 42 de 3977
Jugador 43 de 3977
Jugador 44 de 3977
Jugador 45 de 3977
Jugador 46 de 3977
Jugador 47 de 3977
Jugador 48 de 3977
Jugador 49 de 3977
Jugador 50 de 3977
Jugador 51 de 3977
Jugador 52 de 3977
Jugador 53 de 3977
Ju

Guardamos los datos a sabiendas de que no se van a guardar bien

In [13]:
df_links_mal = pd.DataFrame()
df_links_mal['Links'] = valores_detallados
df_links_mal.to_excel('../data/LinksRegu_Bajadas.xlsx')